Author: Robert Minneker
Goal: Create small training and dev/test set from TUSZ
Description: This script will split edf files into segments of background
and seizure, we then will create a dataset for training 
from these edf file segments and labels

Assumption: TUSZ dataset is downloaded locally

In [1]:
import os
import math
import numpy as np
import mne
from mne.io import read_raw_edf

In [68]:
def preprocess(raw: mne.io.edf.edf.RawEDF):
    samp_freq = raw.info['sfreq']
    nchannels = raw.info['nchan']
    nyquist_freq = int(samp_freq / 2)
    freqs = range(60, nyquist_freq, 60)
    # Notch filter
    notch_filtered = mne.filter.notch_filter(raw._data, samp_freq, freqs)
    # TODO: Bandpass filter
    # TODO: Referencing op
    # Downsampling
    notch_filtered = mne.io.RawArray(notch_filtered, raw.info)
    downsampled = notch_filtered.resample(samp_freq / 2)._data
    # Reshape to have 64 channels, to fit model
    # TODO: Better strat than this later
    num_samples = np.shape(downsampled)[1]
    if (nchannels > 64):
        # pick only the first 64
        downsampled = downsampled[0:64]
    else:
        # Repeat the channels till we get 64
        right_nchannels = np.zeros((64, num_samples))
        num_reps = math.floor(64 / nchannels)
        for i in range(num_reps):
            start_ind = i * nchannels
            end_ind = (i + 1) * nchannels
            # Downsampled end index, for times when 64 isn't
            # a multiple of nchannels
            dend_ind = nchannels
            if (end_ind > 64):
                overflow = end_ind - 64
                end_ind = 64
                dend_ind = nchannels - overflow

            right_nchannels[i * nchannels: (i+1) * nchannels] = downsampled[0:dend_ind]
        downsampled = right_nchannels

    # Needs another reshaping to be 64 * (multiple of 128) * 1
    return downsampled.reshape(64, num_samples, 1)

In [3]:
def load_edf(filename):
    return read_raw_edf(filename, preload=True)

In [4]:
def get_edf_length(raw: mne.io.edf.edf.RawEDF):
    return np.shape(raw._data)[1]/raw.info['sfreq']

In [5]:
tusz_data_path = '/Users/robert/eeg_data'
# train_path = os.path.join(tusz_data_path, 'edf', 'train', '02_tcp_le')
train_path = os.path.join(tusz_data_path, 'edf', 'train')
# dev_path = os.path.join(tusz_data_path, 'edf', 'dev', '02_tcp_le')
dev_path = os.path.join(tusz_data_path, 'edf', 'dev')
ref_path = os.path.join(tusz_data_path, '_DOCS')
ref_dev_file = os.path.join(ref_path, 'ref_dev.txt')
ref_train_file = os.path.join(ref_path, 'ref_train.txt')

In [6]:
ref_train_file

'/Users/robert/eeg_data/_DOCS/ref_train.txt'

In [15]:
counter = 0
for dirname, _, filenames in os.walk(train_path):
    for filename in filenames:
        counter += 1
        if counter > 100:
            break
        print(os.path.join(dirname, filename))

/Users/robert/eeg_data/edf/train/02_tcp_le/.DS_Store
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001.txt
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t004.lbl_bi
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t003.tse_bi
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t002.lbl_bi
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t004.edf
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t004.tse_bi
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t002.edf
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t003.edf
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s001_t002.tse_bi
/Users/robert/eeg_data/edf/train/02_tcp_le/024/00002478/s001_2006_01_31/00002478_s

In [7]:
bckg_cnt = 0
seiz_cnt = 0
bckg_dict = {}
seiz_dict = {}
with open(ref_train_file) as f:
    for line in f:
        file_name, start_time, end_time, label, prob = line.split()
        if 0.0 <= float(start_time) and float(start_time) <= 2147483647 and 0.0 <= float(end_time) and float(end_time) <= 2147483647:
            file_name = file_name.split('.')[0]
            if label == 'bckg':
                bckg_cnt += 1
                curr_times = bckg_dict.get(file_name, [])
                curr_times.append((start_time, end_time))
                bckg_dict.update({file_name: curr_times})
            elif label == 'seiz':
                seiz_cnt += 1
                curr_times = seiz_dict.get(file_name, [])
                curr_times.append((start_time, end_time))
                seiz_dict.update({file_name: curr_times})
print('bckg samples', bckg_cnt)
print('seiz samples', seiz_cnt)

bckg samples 6976
seiz samples 2377


In [8]:
bckg_cnt = 0
seiz_cnt = 0
bckg_dev_dict = {}
seiz_dev_dict = {}
with open(ref_dev_file) as f:
    for line in f:
        file_name, start_time, end_time, label, prob = line.split()
        if 0.0 <= float(start_time) and float(start_time) <= 2147483647 and 0.0 <= float(end_time) and float(end_time) <= 2147483647:
            file_name = file_name.split('.')[0]
            if label == 'bckg':
                bckg_cnt += 1
                curr_times = bckg_dev_dict.get(file_name, [])
                curr_times.append((start_time, end_time))
                bckg_dev_dict.update({file_name: curr_times})
            elif label == 'seiz':
                seiz_cnt += 1
                curr_times = seiz_dev_dict.get(file_name, [])
                curr_times.append((start_time, end_time))
                seiz_dev_dict.update({file_name: curr_times})
print('bckg samples', bckg_cnt)
print('seiz samples', seiz_cnt)

bckg samples 1686
seiz samples 673


In [9]:
from pathlib import Path
import glob
import mne
from mne.io import read_raw_edf

In [125]:
training_subset_dir = '/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/train'
dev_subset_dir = '/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev'

In [154]:
early_stop_counter_bckg = 0
early_stop_counter_seiz = 0
early_stop_value = 5
for f in glob.glob(dev_path + '/**/*.edf', recursive=True):
    if early_stop_counter_bckg >= early_stop_value and early_stop_counter_seiz >= early_stop_value:
        break
    file_name = f.split('/')[-1].split('.')[0]
    if file_name in bckg_dev_dict.keys():
        if early_stop_counter_bckg < early_stop_value:
            times = bckg_dev_dict.get(file_name)
            raw_data = read_raw_edf(f, preload=True)
            for start_time, end_time in times:
                early_stop_counter_bckg += 1
                new_subset_name = os.path.join(dev_subset_dir, file_name + '_start_' 
                                               + start_time + '_end_' + end_time + '_type_' + 'bckg' + '_eeg.fif')
                try:
                    raw_data.save(new_subset_name, picks=['eeg'], 
                                  tmin=float(start_time), tmax=float(end_time), overwrite=True)
                except ValueError:
                    early_stop_counter_bckg -= 1
    
    if file_name in seiz_dev_dict.keys():
        if early_stop_counter_seiz < early_stop_value:
            times = seiz_dev_dict.get(file_name)
            raw = read_raw_edf(f, preload=True)
            for start_time, end_time in times:
                early_stop_counter_seiz += 1
                new_subset_name = os.path.join(dev_subset_dir, file_name + '_start_' 
                                               + start_time + '_end_' + end_time + '_type_' + 'seiz' + '_eeg.fif')
                try:
                    raw_data.save(new_subset_name, picks=['eeg'], 
                                  tmin=float(start_time), tmax=float(end_time), overwrite=True)
                except ValueError:
                    early_stop_counter_seiz -= 1

Extracting EDF parameters from /Users/robert/eeg_data/edf/dev/03_tcp_ar_a/022/00002297/s003_2007_10_23/00002297_s003_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 185599  =      0.000 ...   463.998 secs...
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_0.0000_end_36.0700_type_bckg_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_0.0000_end_36.0700_type_bckg_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_85.1225_end_123.9525_type_bckg_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_85.1225_end_123.9525_type_bckg_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_s

<ipython-input-154-8c037cfb9b83>:16: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_0.0000_end_36.0700_type_bckg_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:16: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_85.1225_end_123.9525_type_bckg_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:16: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/trai

Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_36.0700_end_85.1225_type_seiz_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_123.9525_end_172.0575_type_seiz_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_123.9525_end_172.0575_type_seiz_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_251.9850_end_309.0100_type_seiz_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_251.9850_end_309.0100_type_seiz_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_328.0225_end_343.0575_type_seiz_eeg.fif
Closing /Users/robert

<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_123.9525_end_172.0575_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t001_start_251.9850_end_309.0100_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/

Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t000_start_0.0000_end_1.0000_type_bckg_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t000_start_318.9350_end_328.8650_type_bckg_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t000_start_318.9350_end_328.8650_type_bckg_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t000_start_371.0975_end_378.9125_type_bckg_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t000_start_371.0975_end_378.9125_type_bckg_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00002297_s003_t000_start_422.8350_end_435.0000_type_bckg_eeg.fif
Closing /Users/robert/D

...

<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008460_s001_t000_start_1.0320_end_109.2181_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008460_s001_t000_start_117.2499_end_668.1400_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/tr

Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t000_start_1.0000_end_198.8865_type_seiz_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t000_start_1.0000_end_198.8865_type_seiz_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t000_start_387.5361_end_556.6289_type_seiz_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t000_start_387.5361_end_556.6289_type_seiz_eeg.fif
[done]
Extracting EDF parameters from /Users/robert/eeg_data/edf/dev/01_tcp_ar/085/00008544/s006_2012_01_05/00008544_s006_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 223231  =      0.000 ...   871.996 secs...


<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t000_start_1.0000_end_198.8865_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t000_start_387.5361_end_556.6289_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],


Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t001_start_2.0204_end_130.2229_type_seiz_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t001_start_2.0204_end_130.2229_type_seiz_eeg.fif
[done]
Writing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t001_start_338.8331_end_503.1211_type_seiz_eeg.fif
Closing /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t001_start_338.8331_end_503.1211_type_seiz_eeg.fif
[done]


<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t001_start_2.0204_end_130.2229_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/training/data_subset/dev/00008544_s006_t001_start_338.8331_end_503.1211_type_seiz_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data.save(new_subset_name, picks=['eeg'],
<ipython-input-154-8c037cfb9b83>:30: RuntimeWarning: This filename (/Users/robert/Desktop/School/Grad/481c/seizure-classification/tr

In [10]:
import shutil

In [12]:
## Move some example edf files to desktop
output_dir_train = '/Users/robert/Desktop/train'
output_dir_dev = '/Users/robert/Desktop/dev'
early_stop_counter_bckg = 0
early_stop_counter_seiz = 0
early_stop_value = 5
for f in glob.glob(dev_path + '/**/*.edf', recursive=True):
    if early_stop_counter_bckg >= early_stop_value and early_stop_counter_seiz >= early_stop_value:
        break
    file_name = f.split('/')[-1].split('.')[0]
    if file_name in bckg_dev_dict.keys():
        if early_stop_counter_bckg < early_stop_value:
            times = bckg_dev_dict.get(file_name)
            str_times = ''
            for start_time, end_time in times:
                str_times += 'start:_' + start_time + '_end:_' + end_time + '_'
                early_stop_counter_bckg += 1
            shutil.copy2(f, os.path.join(output_dir_train, file_name + str_times + '_seiz'))
    
    if file_name in seiz_dev_dict.keys():
        if early_stop_counter_seiz < early_stop_value:
            times = seiz_dev_dict.get(file_name)
            str_times = ''
            for start_time, end_time in times:
                str_times += 'start:_' + start_time + '_end:_' + end_time + '_'
                early_stop_counter_seiz += 1
            shutil.copy2(f, os.path.join(output_dir_train, file_name + str_times + '_bckg'))

In [152]:
early_stop_counter_seiz

549

In [158]:
'00008479_s002_t003_start_302.8639_end_311.2539_type_seiz_eeg.fif'.split('.')[-2].split('_')[-2]

'seiz'

In [106]:
print(bckg_dict.get('00010418_s018_t008'))

[('0.0000', '601.0000')]


In [82]:
seiz_dict.popitem()

('00005426_s010_t001', [('39.8525', '148.9950')])

In [ ]:
bin_width = 10000
bin_interval = 5000
num_of_bins = math.floor(((edf_length - (bin_width / 1000)) * 1000)/bin_interval) + 1

training_dict = {}

raw = load_edf(filename)
edf_length = get_edf_length(raw)

# bin width is always a multiple of 1000, so no need to math floor
# But do it anyway to get an int
bin_width_s = math.floor(bin_width / 1000)
bin_int_s = bin_interval / 1000
# Prediction for each bin
for i in range(num_of_bins):
    # start of ith bin in milliseconds: i * bin_interval
    # Of course there are 128 data points in a second
    # So proper formula is i * bin_interval * 128/1000
    # Round down
    bin_start = math.floor(i * bin_int_s * 128)
    # end of ith bin = start of bin + bin width
    # Conversion: start of bin + (bin width * 128/1000)
    bin_end = bin_start + (bin_width_s * 128)
    curr_bin = prepr_edf_data[0:64, bin_start:bin_end]
    # bin_width > 1, but model expects 1 second each
    # Try passing batch of 1 sec
    batched_bins = np.zeros((bin_width_s, 64, 128, 1))
    # Make bin into batch of 1 second slices
    # Model expecting 1 second (128 samples)
    for j in range(bin_width_s):
        batch_start = j * 128
        batch_end = (j + 1) * 128
        batched_bins[j] = curr_bin[0:64, batch_start:batch_end]

    # See documentation: 
    # https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict
    # FYI: second argument is batch_size, default is 32
    # prob = eeg_net.predict(batched_bins, bin_width_s)
#     prob = mlp.predict(batched_bins, bin_width_s)
    

    # TODO: We are making 2nd class, seizure prob, make sure
    # this is reflected when you train model
    predictions = prob[0:bin_width_s, 1]

    # Aggregate (Average) predictions for the batch
    result = np.mean(predictions).astype(float)
    results.append(result)
    results_file.write(str(result) + "\n")

In [14]:
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import Input, Flatten

In [15]:
def simple_binary_classifier(nb_classes, channels=64, samples=128) -> Model:
    input_layer = Input(shape=(channels, samples, 1))
    flatten = Flatten(name='flatten')(input_layer)
    hidden = Dense(nb_classes, name='hidden')(flatten)
    output = Dense(nb_classes, name='output')(hidden)
    softmax = Activation('softmax', name='softmax')(output)
    return Model(inputs=input_layer, outputs=softmax)

In [16]:
## Test model prediction on known seizure file.
# This file has seizures at these time points:
# 00002297_s003_t001

# start/_0.0000_end/_36.0700_

# start/_85.1225_end/_123.9525_

# start/_172.0575_end/_251.9850_

# start/_309.0100_end/_328.0225_

# start/_343.0575_end/_350.0325_

# start/_412.9925_end/_464.0000

# __seiz.edf

model = simple_binary_classifier(nb_classes=2)

In [18]:
model = keras.models.load_model('.')

In [169]:
test_seiz_file = '/Users/robert/Desktop/train/00002297_s003_t001start:_0.0000_end:_36.0700_start:_85.1225_end:_123.9525_start:_172.0575_end:_251.9850_start:_309.0100_end:_328.0225_start:_343.0575_end:_350.0325_start:_412.9925_end:_464.0000__seiz.edf'

In [20]:
import mne

In [170]:
raw = mne.io.read_raw_edf(test_seiz_file, preload=True)

Extracting EDF parameters from /Users/robert/Desktop/train/00002297_s003_t001start:_0.0000_end:_36.0700_start:_85.1225_end:_123.9525_start:_172.0575_end:_251.9850_start:_309.0100_end:_328.0225_start:_343.0575_end:_350.0325_start:_412.9925_end:_464.0000__seiz.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 185599  =      0.000 ...   463.998 secs...


In [28]:
import numpy as np

In [42]:
model.input

<KerasTensor: shape=(None, 64, 128, 1) dtype=float32 (created by layer 'input_3')>

In [107]:
def preprocess_simple(raw: mne.io.edf.edf.RawEDF):
    samp_freq = raw.info['sfreq']
    nchannels = raw.info['nchan']
    nyquist_freq = int(samp_freq / 2)
    freqs = range(60, nyquist_freq, 60)
    # Notch filter
    raw = raw.notch_filter(freqs)
    # Downsampling
    raw = raw.resample(nyquist_freq)
    raw_df = raw.to_data_frame(index='time')
    num_samples = raw_df.shape[0]
    if (nchannels > 64):
        # pick only the first 64
        raw_df = raw_df.iloc[:, 0:64]
    else:
        # Repeat the channels till we get 64
        right_nchannels = np.zeros((64, num_samples))
        num_reps = math.floor(64 / nchannels)
        for i in range(num_reps):
            start_ind = i * nchannels
            end_ind = (i + 1) * nchannels
            # Downsampled end index, for times when 64 isn't
            # a multiple of nchannels
            dend_ind = nchannels
            if (end_ind > 64):
                overflow = end_ind - 64
                end_ind = 64
                dend_ind = nchannels - overflow

            right_nchannels[i * nchannels: (i+1) * nchannels] = raw_df.iloc[:, 0:dend_ind].transpose()
        downsampled = right_nchannels

    # Needs another reshaping to be 64 * (multiple of 128) * 1
    return downsampled.reshape(64, num_samples, 1)

In [108]:
raw = mne.io.read_raw_edf(test_seiz_file, preload=True)

Extracting EDF parameters from /Users/robert/Desktop/train/00002297_s003_t001start:_0.0000_end:_36.0700_start:_85.1225_end:_123.9525_start:_172.0575_end:_251.9850_start:_309.0100_end:_328.0225_start:_343.0575_end:_350.0325_start:_412.9925_end:_464.0000__seiz.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 185599  =      0.000 ...   463.998 secs...


In [109]:
processed_input = preprocess_simple(raw)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)



In [122]:
processed_input[:,0:128,:].shape

(64, 128, 1)

In [113]:
num_seconds = processed_input.shape[1] / 128

In [115]:
int(num_seconds)

725

In [127]:
import tensorflow as tf

In [128]:
test_tensor=tf.convert_to_tensor(processed_input[:, 0:128,:])

In [138]:
tf.reshape(test_tensor, (-1, 64, 128, 1))

<tf.Tensor: shape=(1, 64, 128, 1), dtype=float64, numpy=
array([[[[-15.26435307],
         [-16.1160159 ],
         [ -9.01154901],
         ...,
         [ -1.00182313],
         [ -8.98717361],
         [-15.63158463]],

        [[-18.8892125 ],
         [-19.90151828],
         [-23.17111738],
         ...,
         [ -9.20097722],
         [ -5.89911849],
         [-20.02175558]],

        [[-17.09398973],
         [  7.05097046],
         [ -5.57320324],
         ...,
         [  6.08266514],
         [ -4.29519078],
         [ -4.81812247]],

        ...,

        [[ 77.40467305],
         [-30.38523632],
         [-83.51282317],
         ...,
         [-25.5274739 ],
         [-11.18469777],
         [  0.33092458]],

        [[ 82.79525231],
         [-33.57266637],
         [-88.50405299],
         ...,
         [-20.97503729],
         [ -3.37774183],
         [  2.00649644]],

        [[ 84.60704481],
         [-33.53398633],
         [-82.16511389],
         ...,
         [

In [172]:
dirs = os.listdir('/Users/robert/Desktop/train/')

# This would print all the files and directories
for file in dirs:
    print(file)

.DS_Store
00002297_s003_t001start:_0.0000_end:_36.0700_start:_85.1225_end:_123.9525_start:_172.0575_end:_251.9850_start:_309.0100_end:_328.0225_start:_343.0575_end:_350.0325_start:_412.9925_end:_464.0000__seiz.edf
00002297_s003_t001start:_36.0700_end:_85.1225_start:_123.9525_end:_172.0575_start:_251.9850_end:_309.0100_start:_328.0225_end:_343.0575_start:_350.0325_end:_412.9925__bckg.edf


In [174]:
raw = mne.io.read_raw_edf('/Users/robert/Desktop/train/00002297_s003_t001start:_36.0700_end:_85.1225_start:_123.9525_end:_172.0575_start:_251.9850_end:_309.0100_start:_328.0225_end:_343.0575_start:_350.0325_end:_412.9925__bckg.edf', preload=True)
processed_input = preprocess_simple(raw)

Extracting EDF parameters from /Users/robert/Desktop/train/00002297_s003_t001start:_36.0700_end:_85.1225_start:_123.9525_end:_172.0575_start:_251.9850_end:_309.0100_start:_328.0225_end:_343.0575_start:_350.0325_end:_412.9925__bckg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 185599  =      0.000 ...   463.998 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)



In [162]:
import pickle
import math

In [144]:
nb_model = pickle.load(open('nb_model.sav', 'rb'))

In [160]:
scalerfile = 'scaler.sav'
scaler = pickle.load(open(scalerfile, 'rb'))

In [176]:
for time_idx in range(int(processed_input.shape[1] / 178)):
    start_idx = time_idx * 178
    end_idx = start_idx + 178
    curr_batch = processed_input[:, start_idx:end_idx, :]
#     print(curr_batch.shape)
    curr_batch = np.squeeze(curr_batch, -1)
#     print(curr_batch.shape)
    print(curr_batch)
    curr_batch = scaler.transform(curr_batch)
    print(curr_batch)
    prediction = nb_model.predict_proba(curr_batch)[:,1]
    print('time:', time_idx, 'prediction:', np.mean(prediction))
    break

[[-15.26435307 -16.1160159   -9.01154901 ...  -4.05607114  15.41696436
  -20.90807884]
 [-18.8892125  -19.90151828 -23.17111738 ... -10.67182682   4.66350386
    8.47943746]
 [-17.09398973   7.05097046  -5.57320324 ...   5.81925384   1.77750059
   -6.56948547]
 ...
 [ 77.40467305 -30.38523632 -83.51282317 ...  -6.63231615 -14.64448183
  -14.75638078]
 [ 82.79525231 -33.57266637 -88.50405299 ... -12.090962    -8.96178723
  -17.63018816]
 [ 84.60704481 -33.53398633 -82.16511389 ...  -5.15218976 -10.07019427
  -10.59694285]]
[[-0.02741627 -0.03748     0.00252704 ...  0.05364711  0.17282025
  -0.05188875]
 [-0.04953552 -0.06071171 -0.08599204 ...  0.01285368  0.10656805
   0.12738585]
 [-0.03858089  0.10469632  0.02402199 ...  0.11453939  0.08878735
   0.03558191]
 ...
 [ 0.53805928 -0.12505052 -0.46322054 ...  0.03776172 -0.01238867
  -0.01436114]
 [ 0.57095313 -0.14461185 -0.49442341 ...  0.00410314  0.02262248
  -0.03189242]
 [ 0.58200887 -0.14437447 -0.45479529 ...  0.04688833  0.01579

In [ ]:
tf.reshape((-1,))

In [75]:
raw = raw.notch_filter(np.arange(60, 200, 60))

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)



In [94]:
raw_df = raw.to_data_frame(index='time')

In [95]:
raw_df.shape

(185600, 32)

In [98]:
raw_df.iloc[0:128, 0]

time
0     -15.286270
2     -15.651880
5     -15.332563
8     -13.712253
10    -11.514841
         ...    
308    -8.472839
310   -14.199593
312   -11.262648
315    -5.189014
318   -22.275065
Name: EEG FP1-REF, Length: 128, dtype: float64

In [93]:
raw.info['nchan']

32

In [90]:
raw_df.iloc[0:128, 1]

0     -15.286270
1     -15.651880
2     -15.332563
3     -13.712253
4     -11.514841
         ...    
123    -8.472839
124   -14.199593
125   -11.262648
126    -5.189014
127   -22.275065
Name: EEG FP1-REF, Length: 128, dtype: float64